In [10]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

In [11]:
def show_image(image, title='titlu'):
    # image=cv.resize(image,(0,0),fx=0.5,fy=0.5)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [12]:
def get_game_matrix():
    
    matrix = [
        ['x3', '', '', '', '', '', 'x3', 'x3', '', '', '', '', '', 'x3'],
        ['', 'x2', '', '', '/', '', '', '', '', '/', '', '', 'x2', ''],
        ['', '', 'x2', '', '', '-', '', '', '-', '', '', 'x2', '', ''],
        ['', '', '', 'x2', '', '', '+', '*', '', '', 'x2', '', '', ''],
        ['', '/', '', '', 'x2', '', '*', '+', '', 'x2', '', '', '/', ''],
        ['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
        ['x3', '', '', '*', '+', '', 1, 2, '', '*', '+', '', '', 'x3'],
        ['x3', '', '', '+', '*', '', 3, 4, '', '+', '*', '', '', 'x3'],
        ['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
        ['', '/', '', '', 'x2', '', '+', '*', '', 'x2', '', '', '/', ''],
        ['', '', '', 'x2', '', '', '*', '+', '', '', 'x2', '', '', ''],
        ['', '', 'x2', '', '', '-', '', '', '-', '', '', 'x2', '', ''],
        ['', 'x2', '', '', '/', '', '', '', '', '/', '', '', 'x2', ''],
        ['x3', '', '', '', '', '', 'x3', 'x3', '', '', '', '', '', 'x3']
    ]
    
    return matrix

In [70]:
def get_play_area(image):
    
    # luam canalul blue pentru ca diferenta de intensitate intre masa si tabla e mai mare
    
    blue, _, _ = cv.split(image)
    
    # bagam filtrele din laborator
    
    image_m_blur = cv.medianBlur(blue, 3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    thresh = cv.erode(thresh, np.ones((5, 5), np.uint8))

    # codul din laborator, luam cel mai mare contur care va fi tabla

    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    top_left, top_right, bottom_right, bottom_left = 0, 0, 0, 0
    
    for i in range(len(contours)):
        if len(contours[i]) > 3:
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis=1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
    
    width = 2000
    height = 2000
    
    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype="float32")
    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    # result e tabla (cu tot cu marginea albastra)
    result = cv.warpPerspective(image, M, (width, height))
    
    # hard-coded luam doar partea relevanta din tabla si o facem de 1400 x 1400 ca sa aiba fiecare patratel 100 x 100
    x_start, y_start = 260, 260
    x_end, y_end = 1740, 1750
    game_area = result[y_start:y_end, x_start:x_end]
    resized_game_area = cv.resize(game_area, (1400, 1400))
    
    return resized_game_area


In [94]:
pieces = [x for x in range(22)] + [24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 80, 81, 90]

def classify_number(patch):
        maxi=-np.inf
        poz=-1
        _, patch_thresh = cv.threshold(patch, 67, 255, cv.THRESH_BINARY)
        # am facut poza asta mai mare ca gen daca eu cautam un template intr o poza si ambele aveau aceeasi dimensiune imi rezulta un singur pixel??
        # un singur canal o facem
        picture_for_search = np.full(shape=(patch.shape[0] * 3, patch.shape[1] * 3), dtype=np.uint8, fill_value=255)
        picture_for_search[patch.shape[0]: 2*patch.shape[0], patch.shape[1]:2*patch.shape[1]] = patch_thresh
        cv.imwrite('big.jpg', picture_for_search)

        for piece in pieces:
            img_template = cv.imread('templates/'+str(piece)+'.jpg')
            img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
            _, img_template = cv.threshold(img_template, 67, 255, cv.THRESH_BINARY) 
            corr = cv.matchTemplate(picture_for_search, img_template,  cv.TM_CCOEFF_NORMED)
            corr=np.max(corr)
            if corr>maxi:
                maxi=corr
                poz=piece
        return poz

In [85]:
def plot_heatmap(matrix):
    
    fig, ax = plt.subplots(figsize=(8, 8))
    heatmap = ax.imshow(matrix, cmap='viridis', interpolation='nearest')

    for i in range(14):
        for j in range(14):
            ax.text(j, i, f"{matrix[i, j]:.1f}", ha='center', va='center', color='white')

    plt.colorbar(heatmap, label='Mean Difference')
    ax.set_xticks(range(matrix.shape[1]))
    ax.set_yticks(range(matrix.shape[0]))
    ax.set_xticklabels([chr(65 + j) for j in range(matrix.shape[1])])  
    ax.set_yticklabels(range(1, matrix.shape[0] + 1))  
    plt.title('Mean Difference Heatmap')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.show()

In [89]:
lines_horizontal = []
lines_vertical = []

for i in range(0,1401,100):
    l_h = []
    l_v=[]
    l_h.append((0,i))
    l_h.append((1399,i))
    l_v.append((i,0))
    l_v.append((i,1399))
    lines_horizontal.append(l_h)
    lines_vertical.append(l_v)

def get_placed_piece_position(prev_board, curr_board):
    
    # grayscale + threshhold pe poza previous si current
     
    prev_gray = cv.cvtColor(prev_board,cv.COLOR_BGR2GRAY)
    curr_gray = cv.cvtColor(curr_board,cv.COLOR_BGR2GRAY)
    
    _, prev_board_thresh = cv.threshold(prev_gray, 67, 255, cv.THRESH_BINARY)
    _, curr_board_thresh = cv.threshold(curr_gray, 67, 255, cv.THRESH_BINARY)
    
    # ------------------
    
    searched_i, searched_j = 0, 0
    max_mean_difference = 0
    mean_diffs_matrix = np.zeros((14,14)) # for heatmap
    new_piece_patch = None
    
    # pe fiecare patch facem diferenta absoluta intre media intensitatii pixelilor si luam maximul
    
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 8
            y_max = lines_vertical[j + 1][1][0] - 8
            x_min = lines_horizontal[i][0][1] + 8
            x_max = lines_horizontal[i + 1][1][1] - 8
            prev_patch = prev_board_thresh[x_min:x_max, y_min:y_max].copy()
            curr_patch = curr_board_thresh[x_min:x_max, y_min:y_max].copy()
            
            prev_patch_mean = np.mean(prev_patch)
            curr_patch_mean = np.mean(curr_patch)
            
            mean_diffs_matrix[i][j] = abs(prev_patch_mean - curr_patch_mean)
            
            if abs(prev_patch_mean - curr_patch_mean) > max_mean_difference:
                max_mean_difference = abs(prev_patch_mean - curr_patch_mean)
                searched_i, searched_j = i, j
                new_piece_patch = curr_gray[x_min:x_max, y_min:y_max].copy()
           
    number_on_piece = classify_number(new_piece_patch)
    
    # plot_heatmap(mean_diffs_matrix)

    return (searched_i, searched_j) , number_on_piece

In [95]:
# ------------------ pt documentatie

prev_pic = cv.imread('antrenare/2_25.jpg')
curr_pic = cv.imread('antrenare/2_26.jpg')
prev_board = get_play_area(prev_pic)
curr_board = get_play_area(curr_pic)

a, b = get_placed_piece_position(prev_board, curr_board)

# cv.imwrite('thresh-26.jpg', curr_board_thresh_copy)

# ------------------

In [17]:
operations = ['+', '-', '*', '/']
multipliers = ['x2', 'x3']
directions = [[(-1, 0), (-2, 0)], [(0, 1), (0, 2)], [(1, 0), (2, 0)], [(0, -1), (0, -2)]]

def is_within_bounds(i, j, matrix_size=14):
    return 0 <= i < matrix_size and 0 <= j < matrix_size

def get_move_score(game_matrix, piece, position):
    
    piece_i, piece_j = position
    under_piece = game_matrix[piece_i][piece_j]
    valid_eq = []
    used_multiplier = 1
    
    if under_piece in multipliers:
        used_multiplier = int(under_piece[-1])
    
    for direction in directions:
        neighb_i = piece_i + direction[0][0]
        neighb_j = piece_j + direction[0][1]
        far_neighb_i = piece_i + direction[1][0]
        far_neighb_j = piece_j + direction[1][1]
        
        if is_within_bounds(neighb_i, neighb_j) and is_within_bounds(far_neighb_i, far_neighb_j):
            neighb = game_matrix[neighb_i][neighb_j]
            far_neighb = game_matrix[far_neighb_i][far_neighb_j]
            if type(neighb) is int and type(far_neighb) is int:
                if neighb + far_neighb == piece:
                    valid_eq.append((piece, '+'))
                elif neighb * far_neighb == piece:
                    valid_eq.append((piece, '*'))
                elif max(neighb, far_neighb) - min(neighb, far_neighb) == piece: 
                    valid_eq.append((piece, '-'))
                elif min(neighb, far_neighb) != 0:
                    if max(neighb, far_neighb) // min(neighb, far_neighb) == piece: 
                        valid_eq.append((piece, '/'))
                    
    if under_piece in operations:
        valid_eq = [x for x in valid_eq if x[1] == under_piece]
            
    game_matrix[piece_i][piece_j] = piece
        
    return game_matrix, used_multiplier * sum([x[0] for x in valid_eq])
    


In [28]:
moves = ['0' + str(x) for x in range(1, 10)]  + [str(x) for x in range (10, 51)]

def solve_game(game_idx):
    
    game_matrix = get_game_matrix() # matrix that represents the game table
    turn_vector = [(line.split()[0], int(line.split()[1])) for line in open(f'evaluare/fake_test/{game_idx}_turns.txt')] # vector of tuples of player name and move index
    
    current_turn_score = 0 # score for the player that is currently moving
    current_turn_index = 0 
    current_player = turn_vector[current_turn_index][0]
    
    scores_file = open(f'331_Ingeaua_Alexandru-Cristian/{game_idx}_scores.txt', 'a')
    
    for move_idx in range(len(moves)):
        prev_img = None
        if moves[move_idx] == '01':
            prev_img = cv.imread('imagini_auxiliare/01.jpg')
        else:
            prev_img = cv.imread(f'evaluare/fake_test/{game_idx}_{moves[move_idx-1]}.jpg')
        curr_img = cv.imread(f'evaluare/fake_test/{game_idx}_{moves[move_idx]}.jpg')
        
        prev_board = get_play_area(prev_img)
        curr_board = get_play_area(curr_img)
        
        position, piece = get_placed_piece_position(prev_board, curr_board)
        
        move_file = open(f'331_Ingeaua_Alexandru-Cristian/{game_idx}_{moves[move_idx]}.txt', 'w')
        move_file.write(f'{position[0]+1}{chr(position[1]+65)} {piece}')
        move_file.close()
        
        game_matrix, move_score = get_move_score(game_matrix, piece, position)
        current_turn_score += move_score
        
        if current_turn_index < len(turn_vector)-1:
            if move_idx + 2 == turn_vector[current_turn_index+1][1]:
                scores_file.write(f'{current_player} {turn_vector[current_turn_index][1]} {current_turn_score}\n')
                current_turn_score = 0
                current_turn_index += 1
                current_player = turn_vector[current_turn_index][0] 
        if move_idx == 49:
            scores_file.write(f'{current_player} {turn_vector[current_turn_index][1]} {current_turn_score}')
            return

    scores_file.close()
            

In [29]:
solve_game(1)

In [20]:
os.mkdir('331_Ingeaua_Alexandru-Cristian')

for game in ['1', '2', '3', '4']:
    solve_game(game)
    

In [49]:
# def get_piece_patches(empty_board, board_with_pieces, lines_horizontal, lines_vertical):
# 
#     empty_gray = cv.cvtColor(empty_board, cv.COLOR_BGR2GRAY)
#     full_gray = cv.cvtColor(board_with_pieces, cv.COLOR_BGR2GRAY)
# 
#     _, empty_board_thresh = cv.threshold(empty_gray, 67, 255, cv.THRESH_BINARY)
#     _, full_board_thresh = cv.threshold(full_gray, 67, 255, cv.THRESH_BINARY)
# 
#     # ------------------
# 
#     mean_diffs_abs = np.zeros((14, 14))  # abs diff matrix
# 
#     # ------------------
# 
#     for i in range(len(lines_horizontal) - 1):
#         for j in range(len(lines_vertical) - 1):
#             y_min = lines_vertical[j][0][0] + 8
#             y_max = lines_vertical[j + 1][1][0] - 8
#             x_min = lines_horizontal[i][0][1] + 8
#             x_max = lines_horizontal[i + 1][1][1] - 8
#             empty_patch = empty_board_thresh[x_min:x_max, y_min:y_max].copy()
#             full_patch = full_board_thresh[x_min:x_max, y_min:y_max].copy()
# 
#             empty_patch_mean = np.mean(empty_patch)
#             full_patch_mean = np.mean(full_patch)
# 
#             mean_diffs_abs[i][j] = abs(empty_patch_mean - full_patch_mean)
# 
#     flat_matrix = mean_diffs_abs.flatten()
# 
#     top_indices = np.argpartition(flat_matrix, -46)[-46:]
#     top_indices = top_indices[np.argsort(flat_matrix[top_indices])[::-1]]
#     top_indices_2d = [divmod(idx, 14) for idx in top_indices]
# 
#     patches = []
#     
#     for i, j in top_indices_2d:
#         y_min = lines_vertical[j][0][0] + 8
#         y_max = lines_vertical[j + 1][1][0] - 8
#         x_min = lines_horizontal[i][0][1] + 8
#         x_max = lines_horizontal[i + 1][1][1] - 8
#         searched_patch = board_with_pieces[x_min:x_max, y_min:y_max].copy()
#         patches.append(searched_patch)
#         
#     if not os.path.exists("templates"):
#         os.makedirs("templates")
# 
#     for idx, patch in enumerate(patches):
#         full_path = f"templates/{idx}.jpg"
#         cv.imwrite(full_path, patch)
#         
# empty_board = cv.imread('imagini_auxiliare/01.jpg')
# board_with_pieces = cv.imread('imagini_auxiliare/04.jpg')
# 
# empty_board = get_play_area(empty_board)
# board_with_pieces = get_play_area(board_with_pieces)
# 
# get_piece_patches(empty_board, board_with_pieces, lines_horizontal, lines_vertical)